Job titles recommendation by skills

recommend Job titles according to skills

I used CountVectorizer for feature engineering. 
Then use cosine_similarity between inserted skills(after tranform it by Vectorizer) 
with each Dataset skill row after remove all nan and deplicated rows.

In [1]:
import numpy as np
import pandas as pd

#pre_processing libraries
import string
import re
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

# Import CountVectorizer and TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#read data
data=pd.read_csv('jobs_skills.csv')
# delete some columns
data = data.drop(['jobFunction','industry','_id'],axis=1)
# delete nan rows
data=data[data['skills'] != "['nan']"]
# delete duplicated rows
data=data.drop_duplicates(subset ="skills")
# clean job title by remove everything after (_)
data['title']=data['title'].str.split('-').str[0]

In [3]:
# clean job skills
def clean(row):
    row =row.lower()
    row = re.sub(r'[^\w]', ' ', row)
    lemma_row = " ".join(lemma.lemmatize(word) for word in row.split() 
                         if word not in stopwords )
    return lemma_row

data['skills']= [clean(row) for row in data['skills']]
data.head()

,title,skills
0,Property Consultant,sale retail real estate sale target indoor sal...
1,Sales Representative,sale real estate sale target customer service ...
2,Receptionist,admin work office management administration ad...
3,Senior Property Consultant,sale real estate sale target computer skill co...
4,Senior SEO Specialist,marketing campaign e marketing digital marketi...


Build CountVectorizer vocablary 

In [4]:
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(data['skills'])

In [5]:
#count_matrix has shape of rows= data rows no. and columns= data words no.
count_matrix.shape

(5146, 3340)

In [9]:

def find_similarity(input_skills):
    similarity_list=[]
    recommended_jobs={}
    #clean input_skills
    input_skills=clean(input_skills)
    #transform input_skills by vectorizer
    input_skills=vectorizer.transform([input_skills])
    
    #cosine_similarity between input_skills with data skills
    for i in range(1,data['skills'].shape[0]):
        similarity_list.append(cosine_similarity(count_matrix[i], input_skills))
        
    # creating a Series with the similarity scores in descending order   
    similarity_series = pd.Series(similarity_list).sort_values(ascending = False)
    # getting the indexes of the 10 most similar jobs
    top_10_indexes = list(similarity_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching jobs
    for i in top_10_indexes:
        recommended_jobs[data.iloc[i,0]] = list(set(data['skills'][0])-set(data.iloc[i,1]))
    return recommended_jobs
    
    

In [10]:
similar_job_titles= find_similarity(data['skills'][0])
print(similar_job_titles)


{'Property Consultant': ['n', 'd'], 'Sales Team Leader': ['p', 'o'], 'Purchasing Manager': ['k'], 'Site Engineer': ['p', 'y', 'k'], 'Social Media Specialist': ['y'], 'Office Manager': ['l', 'k'], 'Receptionist ': ['g', 'l', 'k', 'p', 'y'], 'IT Help Desk Specialist': [], 'OD Section Head': ['g', 'l', 'k', 'p', 'y'], 'Real Estate Sales Agent': []}
